In [2]:
import yolov1

yolov1 = yolov1.YOLOV1()
model = yolov1.build_model_vgg16()
model.load_weights('dog_cat_duck.h5')


Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [8]:
import data_loader

test_img_dir = 'D:\\projects_test\\yolov1\\datasets\\dog_cat_duck\\valid '
x_test, y_test = data_loader.load_dataset(test_img_dir, (224,224), 3)

In [9]:
predict = model.predict(x_test[:10])

In [25]:
import numpy as np 
import cv2

total = len(predict)

rows, cols = yolov1.height, yolov1.width 

labels = ['dog', 'cat', 'duck']

for i in range(total):
    img = (x_test[i]).copy()
    features = predict[i]
    for j in range(1):

        if j == 0:
            confidence = features[...,4]
            bbox = features[...,0:4]
            box_color = (0,255,0)
        else:
            confidence = features[...,9]
            bbox = features[...,5:9]
            box_color = (0,0,255)
        cl_list = features[...,10:]
 
        # print()
        # cl = np.argmax(features[10:2])
        count = 0
        for m in range(7):
            for n in range(7):
                if confidence[m,n] > 0.5:
                    cl = labels[np.argmax(cl_list[m,n])]
                    
                    count += 1
                    x = (bbox[m,n,0] + n) /6.
                    y = (bbox[m,n,1] + m) /6.
                    w = bbox[m,n,2] 
                    h = bbox[m,n,3] 

                    pt1 = (int((x - w/2) * cols), int((y - h/2) * rows))
                    pt2 = (int((x + w/2) * cols), int((y + h/2) * rows))
                    cv2.rectangle(img, pt1, pt2, box_color, 1)
                    cv2.putText(img, cl, pt2, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255),1)
                    
    cv2.namedWindow('img', 0)
    cv2.imshow('img', img)
    cv2.waitKey(0)

cv2.destroyAllWindows()

In [12]:
np.argmax(predict[0,:,:,10:])

141